In [ ]:
import tenseal.sealapi as seal
from polynomial import enc_poly, divByPo2
import util
import numpy as np
import matplotlib.pyplot as plt
%matplotlib widget

In [ ]:
parms = seal.EncryptionParameters(seal.SCHEME_TYPE.CKKS)
# How large should plaintexts be? This determines number of available levels for a certain bit size, number of data slots and security. 
poly_modulus_degree = 2**15
# Bit size for primes in modulus chain. Larger -> more available options, slower performance. 
bit_size = 50
# How many data levels do we need?
levels = 10
# Create modulus chain
mod_chain_bit_size = [bit_size for k in range(levels)]
# First and last primes are "special" in SEAL, and should always be 60 bit primes.  
mod_chain_bit_size.insert(0,60)
mod_chain_bit_size.append(60)

# Set parameters (this involves involves finding candidate primes for the modulus chain)
parms.set_poly_modulus_degree(poly_modulus_degree)
parms.set_coeff_modulus(seal.CoeffModulus.Create(poly_modulus_degree, mod_chain_bit_size))
scale = pow(2.0, bit_size)

# What are the specific primes in the modulus chain? These help calculate exact scales 
# of rescaled ciphertexts later on in this notebook. 
primes = [modulus.value() for modulus in parms.coeff_modulus()]

# The creation of a SEAL context does not include a check to see if parameter settings
# are valid. This is done later during key creation. 
context = seal.SEALContext(parms,True,seal.SEC_LEVEL_TYPE.TC128)

util.print_parameters(context)

print("|\t Max Bit Count: " + str(seal.CoeffModulus.MaxBitCount(poly_modulus_degree, seal.SEC_LEVEL_TYPE.TC128)))

In [ ]:

keygen = seal.KeyGenerator(context)
secret_key = keygen.secret_key()

public_key = seal.PublicKey()
keygen.create_public_key(public_key)

relin_keys = seal.RelinKeys()
keygen.create_relin_keys(relin_keys)

galois_keys = seal.GaloisKeys()
keygen.create_galois_keys()

encryptor = seal.Encryptor(context, public_key)
evaluator = seal.Evaluator(context)
decryptor = seal.Decryptor(context, secret_key)

encoder = seal.CKKSEncoder(context)
slot_count = seal.CKKSEncoder.slot_count(encoder)

print('Number of slots: ' + str(slot_count))